In [7]:
import torch 

import torch.nn as nn 

import torch.nn.functional as F

import torchvision

import numpy as np

from  torchvision import datasets, transforms

from torch.utils.data import DataLoader, Subset



In [8]:
# Transform: Convert to tensor and normalize (grayscale optional)

transform = transforms.Compose([

    transforms.ToTensor(),

    transforms.Normalize((0.5),(0.5))
])

In [9]:
# Load Full CIFAR-10

train_set = torchvision.datasets.CIFAR10(root='.data', train=True, download='True',transform=transform)

test_set = torchvision.datasets.CIFAR10(root='.data', train=False, download='True',transform=transform)

In [10]:
# Label map

def filter_dataset(dataset):

    idx = [i for i, (_, label)in enumerate (dataset) if label == 5 or label !=5]

    images = [dataset[i][0] for i in idx]

    labels = [1 if dataset[i][1]==5 else 0 for i in idx] # 1 = dog, 0 = not-dog

    return list(zip(images,labels))

In [11]:
train_filtered = filter_dataset(train_data)

test_filtered = filter_dataset(test_data)

In [12]:
# Wrap in DataLoader

train_loader = DataLoader(train_filtered, batch_size=64, shuffle=True)

test_loader = DataLoader(test_filtered, batch_size=64, shuffle=False)

print(f'Filtered Training Samples: {len(train_filtered)}')

print(f'Filtered Testing Samples: {len(test_filtered)}')

Filtered Training Samples: 50000
Filtered Testing Samples: 10000


CNN Architecture

Conv(1 → 8) → ReLU → MaxPool
Conv(8 → 16) → ReLU → MaxPool
Flatten → FC(16×6×6 → 32) → ReLU → FC(32 → 2)

In [13]:
class DogNet(nn.Module):

    def __init__(self):
        super(DogNet,self).__init__()

        self.conv1 = nn.Conv2d(1, 8 , kernel_size=3, padding=1) # [B, 1, 32, 32] -> [B, 8, 32, 32]

        self.pool = nn.MaxPool2d(2,2)  # -> [B, 8, 16, 16]

        self.conv2 = nn.Conv2d(8,16 , kernel_size=3, padding=1)  # -> [B, 16, 16, 16] → [B, 16, 8, 8]


        self.fc1 = nn.Linear(16 * 8 * 8, 32)

        self.fc2 = nn.Linear(32, 2)  # Output: Dog or Not-Dog



    def forward(self,x):

        x = self.pool(F.relu(self.conv1(x))) # Conv1 + ReLU + Pool

        x = self.pool(F.relu(self.conv2(x)))  # Conv2 + ReLU + Pool

        x = x.view(-1, 16 * 8 * 8)   # Flatten

        x = F.relu(self.fc1(x)) # FC1 + ReLU

        x = self.fc2(x)         # Final FC

        return x

Instantiate the Model

In [14]:

# Define your model

model = DogNet()

# Device setup for Mac M1

if torch.backends.mps.is_available():

    device = torch.device("mps")

    print("Using MPS (Metal GPU on Mac)")

else:

    device = torch.device("cpu")

    print("Using CPU")

model.to(device)

print(model)


Using MPS (Metal GPU on Mac)
DogNet(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1024, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=2, bias=True)
)
